#### RUN THIS BLOCK TO START THE APPLICATION
#### GO TO THE LOCALHOST URL
#### iNTERRUPT THE KERNEL TO STOP

In [1]:
# Importing flask module in the project is mandatory
# An object of Flask class is our WSGI application.
from flask import Flask, render_template, request, send_from_directory
import utils
import train_models as tm
import os
import pandas as pd

# Flask constructor takes the name of
# current module (__name__) as argument.

app = Flask(__name__)

def perform_training(stock_name, df, models_list):
    all_colors = {'SVR_linear': '#FF9EDD',
                  'SVR_poly': '#FFFD7F',
                  'SVR_rbf': '#FFA646',
                  'linear_regression': '#CC2A1E',
                  'random_forests': '#8F0099',
                  'KNN': '#CCAB43',
                  'elastic_net': '#CFAC43',
                  'DT': '#85CC43',
                  'LSTM_model': '#CC7674'}

    print(df.head())
    dates, prices, ml_models_outputs, prediction_date, test_price = tm.train_predict_plot(stock_name, df, models_list)
    origdates = dates
    if len(dates) > 20:
        dates = dates[-20:]
        prices = prices[-20:]

    all_data = []
    all_data.append((prices, 'false', 'Data', '#000000'))
    for model_output in ml_models_outputs:
        if len(origdates) > 20:
            all_data.append(
                (((ml_models_outputs[model_output])[0])[-20:], "true", model_output, all_colors[model_output]))
        else:
            all_data.append(
                (((ml_models_outputs[model_output])[0]), "true", model_output, all_colors[model_output]))

    all_prediction_data = []
    all_test_evaluations = []
    all_prediction_data.append(("Original", test_price))
    for model_output in ml_models_outputs:
        all_prediction_data.append((model_output, (ml_models_outputs[model_output])[1]))
        all_test_evaluations.append((model_output, (ml_models_outputs[model_output])[2]))

    return all_prediction_data, all_prediction_data, prediction_date, dates, all_data, all_data, all_test_evaluations

all_files = utils.read_all_stock_files('individual_stocks_5yr')
# The route() function of the Flask class is a decorator,
# which tells the application which URL should call
# the associated function.
@app.route('/')
# ‘/’ URL is bound with hello_world() function.
def landing_function():
    # all_files = utils.read_all_stock_files('individual_stocks_5yr')
    # df = all_files['A']
    # # df = pd.read_csv('GOOG_30_days.csv')
    # all_prediction_data, all_prediction_data, prediction_date, dates, all_data, all_data = perform_training('A', df, ['SVR_linear'])
    stock_files = list(all_files.keys())

    return render_template('index.html',show_results="false", stocklen=len(stock_files), stock_files=stock_files, len2=len([]),
                           all_prediction_data=[],
                           prediction_date="", dates=[], all_data=[], len=len([]))

@app.route('/process', methods=['POST', 'GET'])
def process():

    #stock_file_name = request.form['stockfile']
    #ml_algoritms = request.form.getlist('mlalgos')

    # all_files = utils.read_all_stock_files('individual_stocks_5yr')
    df = all_files[str(stock_file_name)]
    # df = pd.read_csv('GOOG_30_days.csv')
    all_prediction_data, all_prediction_data, prediction_date, dates, all_data, all_data, all_test_evaluations = perform_training(str(stock_file_name), df, ml_algoritms)
    stock_files = list(all_files.keys())

    return render_template('index.html',all_test_evaluations=all_test_evaluations, show_results="true", stocklen=len(stock_files), stock_files=stock_files,
                           len2=len(all_prediction_data),
                           all_prediction_data=all_prediction_data,
                           prediction_date=prediction_date, dates=dates, all_data=all_data, len=len(all_data))
@app.route('/loading', methods=['POST'])
def loading():
    global stock_file_name
    global ml_algoritms
    stock_file_name = request.form['stockfile']
    ml_algoritms = request.form.getlist('mlalgos')

    return render_template('loading.html', stockfile=stock_file_name, mlalgos=ml_algoritms)

# main driver function
if __name__ == '__main__':
    # run() method of Flask class runs the application
    # on the local development server.
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jun/2022 09:34:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 09:34:39] "GET /static/css/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 404 -
127.0.0.1 - - [21/Jun/2022 09:34:50] "POST /loading HTTP/1.1" 200 -
C:\Users\adila\Documents\stock-price-prediction\utils.py:39: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['date', 'open']), 1, inplace=True)


         date   open    high    low  close  volume Name
0  2013-02-08  69.21  70.700  69.21  70.09  265713  AYI
1  2013-02-11  69.98  70.620  69.98  70.25  204437  AYI
2  2013-02-12  70.32  70.960  69.94  70.01  169035  AYI
3  2013-02-13  70.08  71.140  69.88  71.14  251358  AYI
4  2013-02-14  70.71  71.588  70.71  71.42  290970  AYI


127.0.0.1 - - [21/Jun/2022 09:34:50] "GET /process HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 09:34:50] "GET /static/css/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 404 -
127.0.0.1 - - [21/Jun/2022 09:34:54] "POST /loading HTTP/1.1" 200 -
C:\Users\adila\Documents\stock-price-prediction\utils.py:39: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['date', 'open']), 1, inplace=True)
127.0.0.1 - - [21/Jun/2022 09:34:54] "GET /process HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 09:34:54] "GET /static/css/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 404 -


         date   open
0  2013-02-08  69.21
1  2013-02-11  69.98
2  2013-02-12  70.32
3  2013-02-13  70.08
4  2013-02-14  70.71


127.0.0.1 - - [21/Jun/2022 09:34:59] "POST /loading HTTP/1.1" 200 -
C:\Users\adila\Documents\stock-price-prediction\utils.py:39: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['date', 'open']), 1, inplace=True)
127.0.0.1 - - [21/Jun/2022 09:34:59] "GET /process HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2022 09:34:59] "GET /static/css/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 404 -


         date   open     high    low  close   volume Name
0  2013-02-08  30.67  30.7500  30.51  30.65  3169594  BBT
1  2013-02-11  30.67  30.8000  30.56  30.63  2331836  BBT
2  2013-02-12  30.66  30.6905  30.40  30.60  3294416  BBT
3  2013-02-13  30.63  30.6600  30.40  30.52  2954293  BBT
4  2013-02-14  30.42  30.5600  30.28  30.36  3035758  BBT
